<a href="https://colab.research.google.com/github/khp53/Credit-Card-Fraud-Detection-SecureBoost/blob/main/Credit_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install sdv

In [12]:
import pandas as pd
from sdv.single_table import CTGANSynthesizer
from sdv.evaluation.single_table import run_diagnostic
import numpy as np
import datetime as dt


# Load or create a dataset
real_data = pd.read_csv('creditcard.csv')

# We will need additional metadata as this dataset does not include much valuable metadata
# For PCA complience all of them are encrypted.

# External Metadata
merchant_category = ["Grocery", "Electronics", "Clothing", "Travel", "Restaurants"]
real_data["Merchant Category"] = np.random.choice(merchant_category, size=len(real_data))

device_type = ["Mobile", "Desktop", "Tablet"]
real_data["Device Type"] = np.random.choice(device_type, size=len(real_data))

locations = ["City A", "City B", "City C"]
real_data["Location"] = np.random.choice(locations, size=len(real_data))

# HEre we are using the existing time values from our real_data to calculate a time stamp for metadata
timestamp_start = dt.datetime(2024, 11, 14)
real_data["Transaction Timestamp"] = real_data["Time"].apply(lambda x: timestamp_start + dt.timedelta(seconds=x))

payment_methods = ["Credit Card", "Debit Card", "Apple Pay", "Google Pay", "Interact"]
real_data["Payment Method"] = np.random.choice(payment_methods, size=len(real_data))

channel = ["Online", "In-Store", "POS"]
real_data["Channel"] = np.random.choice(channel, size=len(real_data))

season = ["Winter", "Spring", "Summer", "Holiday"]
real_data["Season"] = np.random.choice(season, size=len(real_data))

card_types = ["Visa", "MasterCard", "American Express", "Discover"]
real_data["Card Type"] = np.random.choice(card_types, size=len(real_data))

real_data.to_csv("augmented_creditcard.csv", index=False)

In [13]:
from sdv.metadata import SingleTableMetadata

augmented_data = pd.read_csv("augmented_creditcard.csv")

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(augmented_data)

print(metadata.to_dict())

# Save metadata to json for later reading
metadata.save_to_json("creditcard_metadata.json")
print("Metadata saved!")

{'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1', 'columns': {'Time': {'sdtype': 'numerical'}, 'V1': {'sdtype': 'numerical'}, 'V2': {'sdtype': 'numerical'}, 'V3': {'sdtype': 'numerical'}, 'V4': {'sdtype': 'numerical'}, 'V5': {'sdtype': 'numerical'}, 'V6': {'sdtype': 'numerical'}, 'V7': {'sdtype': 'numerical'}, 'V8': {'sdtype': 'numerical'}, 'V9': {'sdtype': 'numerical'}, 'V10': {'sdtype': 'numerical'}, 'V11': {'sdtype': 'numerical'}, 'V12': {'sdtype': 'numerical'}, 'V13': {'sdtype': 'numerical'}, 'V14': {'sdtype': 'numerical'}, 'V15': {'sdtype': 'numerical'}, 'V16': {'sdtype': 'numerical'}, 'V17': {'sdtype': 'numerical'}, 'V18': {'sdtype': 'numerical'}, 'V19': {'sdtype': 'numerical'}, 'V20': {'sdtype': 'numerical'}, 'V21': {'sdtype': 'numerical'}, 'V22': {'sdtype': 'numerical'}, 'V23': {'sdtype': 'numerical'}, 'V24': {'sdtype': 'numerical'}, 'V25': {'sdtype': 'numerical'}, 'V26': {'sdtype': 'numerical'}, 'V27': {'sdtype': 'numerical'}, 'V28': {'sdtype': 'numerical'}, 'Amount': {'sdtype': 'n